In [ ]:
# библиотека для транслитерации
!pip install iuliia

In [ ]:
import pandas as pd
import iuliia  # библиотека для транслитерации

def split_fio_to_english(fio_series):
    """
    Функция для разделения ФИО и транслитерации
    """
    # временные DataFrame для результатов
    result = pd.DataFrame(index=fio_series.index)

    for idx, fio in fio_series.items():
        # ФИО на части
        parts = str(fio).strip().split()

        # Обрабатываем разные случаи
        if len(parts) == 3:
            # Стандартный случай: Фамилия Имя Отчество
            last_name, first_name, middle_name = parts
        elif len(parts) == 2:
            # Только Фамилия и Имя
            last_name, first_name = parts
            middle_name = ''
        elif len(parts) > 3:
            # Сложные ФИО, берем первые три части
            last_name, first_name, middle_name = parts[0], parts[1], ' '.join(parts[2:])
        else:
            # Неправильный формат
            last_name, first_name, middle_name = fio, '', ''

        # Транслитерируем каждую часть
        result.loc[idx, 'last_name_en'] = iuliia.ICAO_DOC_9303.translate((last_name)).upper()
        result.loc[idx, 'first_name_en'] = iuliia.ICAO_DOC_9303.translate((first_name)).upper()
        result.loc[idx, 'middle_name_en'] = iuliia.ICAO_DOC_9303.translate((middle_name)).upper()

    return result

def process_fio_dataframe(df, fio_column):
    """
    Основная функция для обработки DataFrame
    """
    # Создаем копию DataFrame
    result_df = df.copy()

    # Разделяем ФИО и транслитерируем
    fio_parts = split_fio_to_english(result_df[fio_column])

    # Добавляем новые колонки к исходному DataFrame
    result_df['last_name_en'] = fio_parts['last_name_en']
    result_df['first_name_en'] = fio_parts['first_name_en']
    result_df['middle_name_en'] = fio_parts['middle_name_en']

    # Удаляем исходную колонку
    # result_df = result_df.drop(columns=[fio_column])

    return result_df

# датасет с известной дата рождения, транслитом и 5 столбцами
sirena = pd.read_csv('sirena_export.csv')
sirena = sirena[['PaxName', 'PaxBirthDate', 'TravelDoc', 'card_number', 'Flight']]
print(sirena.head)
new_sirena = process_fio_dataframe(sirena, "PaxName")
new_sirena['card_number'] = new_sirena['card_number'].str.slice(3)
print(new_sirena.head)
new_sirena.to_csv('new_sirena.csv')

<bound method NDFrame.head of                                  PaxName PaxBirthDate    TravelDoc  \
0               ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ   1999-05-15  9375 053270   
1             КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ          NaN  2244 645520   
2        ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА          NaN  8115 961316   
3               ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ   1999-05-02   98 6865148   
4            НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ   1990-12-26  4396 926588   
...                                  ...          ...          ...   
155752  ЕМЕЛЬЯНОВ МИРОСЛАВ СВЯТОСЛАВОВИЧ          NaN  3511 319326   
155753        ТАРАСОВ АРТЕМИЙ ГЕРМАНОВИЧ          NaN  2199 078129   
155754        ГОРОХОВ БОРИС СВЯТОГОРОВИЧ   1970-10-28  7351 760618   
155755     ДОРОФЕЕВ РАМИЛЬ СВЯТОСЛАВОВИЧ   1994-02-14  3256 077151   
155756      ШАПОВАЛОВА ВАРВАРА ДАНИЛОВНА          NaN  8328 780960   

            card_number  Flight  
0        FF#SU 38116280  SU1306  
1       FF#FB 284903754  SU1323  
2                   NaN  SU

In [ ]:
import pandas as pd
import numpy as np

# датасет с известной дата рождения, транслитом и 5 столбцами
# объединение данных о пользователе
sirena = pd.read_csv('new_sirena.csv')
grouped = sirena.groupby(['last_name_en', 'first_name_en', 'middle_name_en']).agg({
    'TravelDoc': lambda x: ', '.join(pd.Series(x).dropna().unique()),
    'card_number': lambda x: ', '.join(x.dropna().unique()),
    'PaxBirthDate': lambda x: x.dropna().iloc[0] if x.dropna().any() else pd.NaT
}).reset_index()

# отстой строка ниже показала что у 1 человека может быть несколько карт
rows_with_multiple_cards = grouped[grouped['card_number'].str.contains(',', na=False)]
print(rows_with_multiple_cards)

grouped.to_csv('grouped_sirenа.csv')
print(grouped.head)

# проверка что строки нормально слились
grouped_travel_doc_counts = grouped['TravelDoc'].str.split(', ').explode().value_counts()
matching_docs_grouped = grouped_travel_doc_counts[grouped_travel_doc_counts > 1].index

rows_with_matching_docs_grouped = grouped[grouped['TravelDoc'].str.contains('|'.join(matching_docs_grouped), na=False)]
print(rows_with_matching_docs_grouped)

      last_name_en first_name_en middle_name_en                TravelDoc  \
4          ABRAMOV         ANTON     DEMIDOVICH              7143 491318   
12         ABRAMOV        EDUARD     IGNATOVICH  6408 293074, 91 9211640   
25         ABRAMOV    KONSTANTIN     AIDAROVICH  46 0243403, 0551 818846   
46         ABRAMOV         SAMIR     DEMIDOVICH              7514 528566   
49         ABRAMOV        STEPAN      ADELEVICH              8747 283374   
...            ...           ...            ...                      ...   
72690       ZYKOVA         MAIIA       IVANOVNA              4244 091657   
72700       ZYKOVA      NATALIIA      ARSENEVNA              1471 641154   
72702       ZYKOVA      NATALIIA      NAZAROVNA  37 6911794, 0974 927924   
72703       ZYKOVA      NATALIIA  STANISLAVOVNA              0885 887810   
72719       ZYKOVA        ULIANA      ROMANOVNA              2179 110245   

                      card_number PaxBirthDate  
4      SU 902323793, DT 815180054     

In [ ]:
import pandas as pd

# проверка на пересечения prog_name+prog_num from forum_users и card_number from grouped_sirenа_birth
# пересечений нет :(
forum_users = pd.read_csv('forum_users.csv')
forum_users['full_card_number'] = str(forum_users['prog_name']) + ' ' + str(forum_users['prog_num'])

sirena = pd.read_csv('grouped_sirenа_birth.csv')
common_names = forum_users[forum_users['full_card_number'].isin(sirena['card_number'])]
print(common_names)

Empty DataFrame
Columns: [Unnamed: 0, nick, sex, firstname, lastname, status, prog_name, prog_num, full_card_number]
Index: []


In [ ]:
import pandas as pd

pointz = pd.read_csv('pointz_aggregator.csv')
sirena = pd.read_csv('grouped_sirenа.csv')
sirena = sirena.drop('Unnamed: 0', axis=1)
# print(sirena.head)

pointz_agg = pointz.groupby(['uid', 'firstname', 'lastname']).agg({
    'card_number': lambda x: ', '.join(pd.Series(x).dropna().unique()),
    'card_bonus': lambda x: ', '.join(x.dropna().unique())
}).reset_index()
print(pointz_agg.head)

# объединение pointz и сирены с фио+др, добавление поля card_bonus для тех у кого card_number известен

merged = pd.merge(
    sirena,
    pointz_agg[['lastname', 'firstname', 'card_number', 'card_bonus']],
    left_on=['last_name_en', 'first_name_en', 'card_number'],
    right_on=['lastname', 'firstname', 'card_number'],
    how='left'
)

print(merged.head)
print(merged[merged['card_bonus'].notna()])
merged = merged.drop('lastname', axis=1)
merged = merged.drop('firstname', axis=1)
print(merged.head)
merged.to_csv('premerged_sirena.csv')

<bound method NDFrame.head of              uid  firstname    lastname  \
0      100056827     ARIANA     SYCHEVA   
1      100068002      RINAT     KULIKOV   
2      100096293    VALERIY   SAVITSKIY   
3      100145156     ALBINA  VORONTCOVA   
4      100157674      ELENA  GORBACHEVA   
...          ...        ...         ...   
56938  999861164    VARVARA   VOLOSHINA   
56939  999891790  VLADISLAV   KAZANTSEV   
56940  999920175    MARSEL'     FEDOSOV   
56941  999947177      NAZAR   FILIMONOV   
56942  999982079      DAVID       ZYKOV   

                                   card_number  \
0                                 DT 397973837   
1                                 FB 876611951   
2                                 DT 654273077   
3      FB 417903001, DT 13179451, SU 752979446   
4                    DT 82333028, SU 487934754   
...                                        ...   
56938                             DT 374754921   
56939                              SU 63265138   
569

In [ ]:
import pandas as pd
from datetime import datetime

# проход по всем строкам в pointz если эти люди есть в boarding data(фи+номер рейса+дата рейса)
# достаем данные о них из boarding_data и пытаемся их найти в сирене (фио+др) и записываем card_number если нашли
# таблица тех кто есть в pointz и нет в boarding data - temp_nobd_pointz.csv

# очень долго, около часа

# Загружаем данные
pointz = pd.read_csv('pointz_aggregator.csv')
pointz = pointz.drop_duplicates(
        subset=['firstname', 'lastname', 'card_number'],
        keep='first'
    )
print(pointz.head)
boarding_data = pd.read_csv('boarding_data.csv', sep=';')
sirena = pd.read_csv('premerged_sirena.csv')


# Преобразуем PassengerBirthDate в boarding_data в формат yyyy-mm-dd
boarding_data['PassengerBirthDate'] = pd.to_datetime(boarding_data['PassengerBirthDate'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

temp_data = [] # нашлись в boarding_data
temp_no_match_data = [] # не нашлись в boarding_data

# Проходим по каждой строке в pointz
for idx, p_row in pointz.iterrows():
    # Ищем совпадения в boarding_data
    matches = boarding_data[
        (boarding_data['PassengerLastName'] == str(p_row['lastname']).strip()) &
        (boarding_data['PassengerFirstName'] == str(p_row['firstname']).strip()) &
        (boarding_data['FlightNumber'] == p_row['activity_code']) &
        (boarding_data['FlightDate'] == p_row['activity_date'])
    ]

    if len(matches) > 0:
        # Добавляем в temp (совпадения найдены)
        for _, b_row in matches.iterrows():
            temp_data.append({
                'lastname': p_row['lastname'],
                'firstname': p_row['firstname'],
                'PassengerSecondName': b_row['PassengerSecondName'],
                'PassengerBirthDate': b_row['PassengerBirthDate'],
                'card_number': p_row['card_number'],
                'FlightNumber': b_row['FlightNumber'],
                'activity_code': p_row['activity_code'],
                'activity_date': p_row['activity_date']
            })
    else:
        # Добавляем в temp_no_match (совпадений не найдено)
        temp_no_match_data.append({
            'lastname': p_row['lastname'],
            'firstname': p_row['firstname'],
            'activity_code': p_row['activity_code'],
            'activity_date': p_row['activity_date'],
            'card_number': p_row['card_number'],
            'reason': 'No match in boarding_data'
        })

    # Прогресс
    if (idx + 1) % 1000 == 0:
        print(f"Обработано {idx + 1} записей из pointz, найдено {len(temp_data)} совпадений")

# Создаем DataFrame temp
temp = pd.DataFrame(temp_data)
temp_no_match = pd.DataFrame(temp_no_match_data)
temp.to_csv('temp_bd_pointz.csv')
temp_no_match.to_csv('temp_nobd_pointz.csv')
print(f"\nСоздана временная таблица temp с {len(temp)} записями")

print(temp.head)



<bound method NDFrame.head of         Unnamed: 0        uid  firstname   lastname   card_number  \
0                0  613142142    IAROMIR     ZVEREV  FB 171388778   
2                2  103197717   VITALINA   KOROVINA  KE 696768759   
6                6  138879468      DANIL    VAVILOV  DT 980250352   
10              10  138879468      DANIL    VAVILOV  SU 510085965   
19              19  138879468      DANIL    VAVILOV  FB 650748133   
...            ...        ...        ...        ...           ...   
436068      436068  280456697  STEFANIIA  BONDAREVA   FB 18721601   
436077      436077  293103570       EMIL    KAZAKOV  KE 637084941   
436081      436081  293103570       EMIL    KAZAKOV  DT 269347755   
436086      436086  293103570       EMIL    KAZAKOV  SU 856232932   
436088      436088  265781820    UL'YANA   KONONOVA  DT 302785701   

                card_bonus activity_code activity_date activity_departure  \
0              Flying Blue         KE827    2017-08-06          

FileNotFoundError: [Errno 2] No such file or directory: 'boarding_data.csv'

In [ ]:
import pandas as pd

# объединение temp_bd_pointz и сирены

def update_sirena_card_numbers(sirena_df, temp_df):
    """
    Сопоставляет temp_data с sirena и дополняет card_number
    """
    # Создаем копии чтобы не изменять оригиналы
    sirena_clean = sirena_df.copy()
    temp_clean = temp_df.copy()

    # Выполняем merge для нахождения совпадений(дополняется только если есть фио+др в обеих таблицах)
    merged = sirena_clean.merge(
        temp_clean[['lastname', 'firstname', 'PassengerSecondName', 'PassengerBirthDate', 'card_number']],
        left_on=['last_name_en', 'first_name_en', 'middle_name_en', 'PaxBirthDate'],
        right_on=['lastname', 'firstname', 'PassengerSecondName', 'PassengerBirthDate'],
        how='inner',
        suffixes=('', '_temp')
    )

    # Обновляем card_number в sirena
    for idx, row in merged.iterrows():
        # Проверяем, что все ключевые поля не пустые (избегаем NaN совпадений)
        if (pd.isna(row['lastname']) or pd.isna(row['firstname']) or
            pd.isna(row['PassengerSecondName']) or pd.isna(row['PassengerBirthDate'])):
            continue

        current_card = str(row['card_number'])
        new_card = str(row['card_number_temp'])

        # Пропускаем если нет нового card_number или он пустой
        if pd.isna(new_card) or new_card=='NaN' or new_card == '' or new_card == 'nan':
            continue

        # Если в sirena card_number пустой
        if pd.isna(current_card) or current_card=='NaN' or current_card == '' or current_card == 'nan':
            sirena_clean.loc[idx, 'card_number'] = new_card

        # Если card_number уже есть, проверяем что нового значения нет в списке
        else:
            existing_cards = [card.strip() for card in current_card.split(',')]
            if new_card not in existing_cards:
                sirena_clean.loc[idx, 'card_number'] = current_card + ',' + new_card

    return sirena_clean

temp = pd.read_csv('temp_bd_pointz.csv')
sirena = pd.read_csv('premerged_sirena.csv')

updated_temp_df = update_sirena_card_numbers(sirena, temp)
updated_temp_df = updated_temp_df.drop('Unnamed: 0', axis=1)
print(updated_temp_df.head)
updated_temp_df.to_csv('merged_sirena.csv')
print(updated_temp_df[updated_temp_df['card_bonus'].unique()])

FileNotFoundError: [Errno 2] No such file or directory: 'temp_bd_pointz.csv'

In [ ]:
import pandas as pd
updated_temp_df = pd.read_csv('merged_sirena.csv')
print(updated_temp_df['card_bonus'].unique())

[nan 'Aeroflot Bonus' 'Flying Blue' 'Delta SkyMiles' 'Korean Air SKYPASS'
 'Delta SkyMiles, Korean Air SKYPASS' 'Delta SkyMiles, Flying Blue'
 'Aeroflot Bonus, Korean Air SKYPASS' 'Aeroflot Bonus, Delta SkyMiles'
 'Flying Blue, Aeroflot Bonus' 'Flying Blue, Delta SkyMiles'
 'Aeroflot Bonus, Korean Air SKYPASS, Flying Blue'
 'Aeroflot Bonus, Flying Blue' 'Delta SkyMiles, Aeroflot Bonus'
 'Flying Blue, Korean Air SKYPASS' 'Korean Air SKYPASS, Aeroflot Bonus'
 'Korean Air SKYPASS, Flying Blue' 'Korean Air SKYPASS, Delta SkyMiles'
 'Korean Air SKYPASS, Delta SkyMiles, Aeroflot Bonus'
 'Flying Blue, Aeroflot Bonus, Delta SkyMiles'
 'Delta SkyMiles, Korean Air SKYPASS, Flying Blue'
 'Flying Blue, Korean Air SKYPASS, Aeroflot Bonus'
 'Delta SkyMiles, Aeroflot Bonus, Korean Air SKYPASS, Flying Blue'
 'Korean Air SKYPASS, Flying Blue, Aeroflot Bonus'
 'Flying Blue, Korean Air SKYPASS, Delta SkyMiles'
 'Delta SkyMiles, Flying Blue, Korean Air SKYPASS'
 'Korean Air SKYPASS, Flying Blue, Delta Sky

In [ ]:
# маленькие итоги
# есть выгрузки:
# merged_sirena - выгрузка у кого известно фио, из pointz для них добавлены card_number(сильно далеко не у всех он есть)
# temp_bd_pointz - есть в pointz и в boarding_data
# temp_nobd_pointz - есть в pointz, нет в boarding_data

# обзор новых файлов(мало ли пригодиться)
# grouped_sirena - данные из сирены с переведенным фио и объединенными полями по TravelDoc, card_number
# merged_sirena - объединение temp_bd_pointz и сирены(только строки которые есть и там и там)
# new_sirena - данные из сирены с переведенным фио
# premerged_sirena - grouped_sirena с добавленным card_bonus из pointz_aggregator

# temp_bd_pointz - временная таблица данные из pointz и boarding_data
# temp_nobd_pointz - таблица тех кто есть в pointz и нет в boarding data

merged_sirena
last_name_en - фамилия транслит(не уникальное, на основе sirena)
first_name_en - имя транслит(не уникальное, на основе sirena)
middle_name_en - отчество транслит(не уникальное, на основе sirena)
TravelDoc - документы(почти уникальное, может быть несколько, раздление через , в колонке) на основе sirena
card_number - номер карточки(вроде уникальное, мб nan, у 1 пользователя мб несколько карт, сводка pointz+sirena)
PaxBirthDate - дата рождения(мб nan, на основе sirena, yyyy-mm-dd)    
card_bonus - авиакомпании(не уникально, мб nan, у 1 пользователя мб несколько карт, сводка pointz+sirena, раздление через , в колонке)

код программы(первые 2 символа card_number) соответствуют авикомпании
FB - Flying Blue
KE - Korean Air SKYPASS
DT - Delta SkyMiles
SU - Aeroflot Bonus